In [1]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
from dalle_vqvae.classification_unofficial import VQVAE_Conv
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
import pandas as pd 

torch.set_num_threads(32)
torch.manual_seed(911) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gustmd0121. Use `wandb login --relogin` to force relogin


In [47]:
dataset = "peak"
vqvae_model = "/home/hschung/saved_models/timeseries/peak2/model_2990.pt"
model_type = "cnn_transformer"
classification_model = "/home/hschung/saved_models/timeseries/peak_clas_cnn_transf/classification_80.pt"

#ECG Dataset
net = VQVAE_Conv(
    n_emb = 64,
    num_classes = 2,
    vqvae_model = vqvae_model,
    positions = 0,
    mask = 0,
    auc_classification = False,
    model_type = model_type
)
  
#Classification Model for LIME training
net.load_state_dict(torch.load(classification_model)["model_state_dict"])

#params
for param in net.parameters():
    param.requires_grad = False 

if dataset == "flat":
    class0_pth = '/home/smjo/xai/timeseries_xai/data/made_data/amplitude_class0.csv'
    class1_pth = '/home/smjo/xai/timeseries_xai/data/made_data/amplitude_class1.csv'
    class0_data = torch.tensor(pd.read_csv(class0_pth,skiprows=0).values)
    class1_data = torch.tensor(pd.read_csv(class1_pth,skiprows=0).values)
    class0_label = torch.zeros(len(class0_data))
    class1_label = torch.ones(len(class1_data))

    data = torch.cat((class0_data,class1_data), dim=0)
    labels = torch.cat((class0_label, class1_label), dim=0)

elif dataset == "peak":
    class0_pth = '/home/smjo/xai/timeseries_xai/data/made_data/class0.csv'
    class1_pth = '/home/smjo/xai/timeseries_xai/data/made_data/class1.csv'
    class0_data = torch.tensor(pd.read_csv(class0_pth,skiprows=0).values)
    class1_data = torch.tensor(pd.read_csv(class1_pth,skiprows=0).values)
    class0_label = torch.zeros(len(class0_data))
    class1_label = torch.ones(len(class1_data))
    
    data = torch.cat((class0_data,class1_data), dim=0)
    labels = torch.cat((class0_label, class1_label), dim=0)    

elif dataset == "hard_mitbih":
    n_set = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/n_data.pt'))
    n_label = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/n_labels.pt'))
    s_set = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/s_data.pt'))
    s_label = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/s_labels.pt'))
    v_set = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/v_data.pt'))[:400]
    v_label = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/v_labels.pt'))[:400]-2
    f_set = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/f_data.pt'))[:400]
    f_label = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/f_labels.pt'))[:400]-2
    q_set = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/q_data.pt'))
    q_label = torch.tensor(torch.load('/home/hschung/xai_workshop/timeseries/mit_bih_dataset/q_labels.pt'))-3

    ecg_train = torch.cat((v_set, f_set), dim=0)
    labels = torch.cat((v_label, f_label), dim=0)

    data = ecg_train
    
    print(f"X shape:{data.shape}, label shape:{labels.shape}")  

class ECGDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
ds = ECGDataset(data, labels)

# Split the dataset into training, validation, and test sets
train_size = int(0.8 * len(ds))
val_size = int(0.1 * len(ds))
test_size = len(ds) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(ds, [train_size, val_size, test_size])
training_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, pin_memory=True)
validation_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, pin_memory=True)

## Define Model Input and Output 

In [48]:
# test dataset information 
ig_misc = torch.load("/home/hschung/xai_workshop/timeseries/integrated_gradients/peak_dataset_IG_cnn_transformer.pt")
codebook_tokens = ig_misc["codebook_tokens"]
input = ig_misc["input"]
output = ig_misc["output"]
recon = ig_misc["recon"]
labels = ig_misc["labels"]
data = ig_misc["data"]

labels = labels.squeeze(0)

print(codebook_tokens.shape)
print(input.shape)
print(output.shape)
print(recon.shape)
print(labels.shape)
print(data.shape)

torch.Size([142, 13])
torch.Size([142, 13, 64])
torch.Size([142, 2])
torch.Size([142, 1, 208])
torch.Size([142])
torch.Size([142, 1, 208])


In [49]:
net = net.to(device)
model_input = net.embedding #embedding layer output (as model_input)
model_input, codebook_tokens = model_input.to(device), codebook_tokens.to(device)
embedded = model_input(codebook_tokens)
embedded.shape

# Define model output
def model_output(inputs):
  return net.ig(inputs)[0][:,0] #[1,4] -> [4] get the fc-layer output from embedded input

## Instantiate Integrated Gradients Method 

In [50]:
from captum.attr import LayerIntegratedGradients
lig = LayerIntegratedGradients(model_output, model_input)
print(model_output)
print(model_input)

<function model_output at 0x7ff644696dc0>
Embedding(64, 64)


## Divide testset samples into classes

In [51]:
total_codebook_tokens = []
for i in range(len(labels)):
    total_codebook_tokens.append(codebook_tokens[i])

In [52]:
codebook_tokens= torch.stack(total_codebook_tokens)

## Construct Original and Baseline Input 

In [53]:
#select class
baseline_tokens = torch.full(codebook_tokens.shape, 63, device=device)
codebook_tokens = codebook_tokens.to(device)
print(f'original input: {codebook_tokens}')
print(f'baseline input: {baseline_tokens}')

original input: tensor([[63, 63, 63,  ..., 28,  7, 63],
        [53, 53, 53,  ..., 45, 10, 53],
        [59, 59, 59,  ..., 61, 10, 59],
        ...,
        [41, 18, 15,  ...,  0,  0,  0],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [19, 19, 19,  ..., 34, 37, 19]], device='cuda:0')
baseline input: tensor([[63, 63, 63,  ..., 63, 63, 63],
        [63, 63, 63,  ..., 63, 63, 63],
        [63, 63, 63,  ..., 63, 63, 63],
        ...,
        [63, 63, 63,  ..., 63, 63, 63],
        [63, 63, 63,  ..., 63, 63, 63],
        [63, 63, 63,  ..., 63, 63, 63]], device='cuda:0')


## Compute Attributions

In [54]:
print(codebook_tokens.shape)
print(baseline_tokens.shape)
baseline_tokens = baseline_tokens.to(device)
attributions, delta = lig.attribute(inputs = codebook_tokens, baselines = baseline_tokens, return_convergence_delta=True)

print(attributions)

torch.Size([142, 13])
torch.Size([142, 13])
tensor([[[-0.0000e+00, -0.0000e+00,  0.0000e+00,  ..., -0.0000e+00,
           0.0000e+00, -0.0000e+00],
         [ 0.0000e+00, -0.0000e+00,  0.0000e+00,  ..., -0.0000e+00,
          -0.0000e+00, -0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -0.0000e+00,
          -0.0000e+00,  0.0000e+00],
         ...,
         [ 1.7465e-03, -1.2826e-03, -7.9215e-05,  ...,  2.8288e-03,
          -3.8508e-03,  1.6124e-04],
         [ 9.1765e-04,  1.5992e-03,  1.0286e-03,  ..., -1.0455e-03,
           3.1748e-04, -1.3439e-02],
         [-0.0000e+00, -0.0000e+00, -0.0000e+00,  ...,  0.0000e+00,
          -0.0000e+00, -0.0000e+00]],

        [[ 3.3051e-04,  2.3044e-04, -1.7836e-04,  ...,  5.3211e-04,
           1.9982e-04,  1.9137e-03],
         [ 2.1213e-03, -2.6549e-04, -1.9601e-04,  ...,  1.6951e-03,
           3.2032e-03,  5.4400e-03],
         [ 2.9070e-03, -1.2844e-03, -3.3254e-04,  ...,  1.0946e-03,
          -1.4643e-03,  4.9250e-

## Compute Attribution for Each Token

In [55]:
def summarize_attributions(attributions):

    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    
    return attributions

attributions_sum = summarize_attributions(attributions) #[1, 6, 768] -> [6]
print(attributions_sum.size())
print(attributions_sum)

torch.Size([142, 13])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0303, -0.0167,  0.0000],
        [ 0.0116,  0.0194,  0.0046,  ..., -0.0285,  0.0213,  0.0077],
        [ 0.0107, -0.0007, -0.0027,  ..., -0.0046, -0.0071,  0.0087],
        ...,
        [ 0.0104,  0.0003, -0.0574,  ...,  0.0189,  0.0187,  0.0141],
        [ 0.0090, -0.0080, -0.0073,  ...,  0.0040,  0.0035,  0.0035],
        [ 0.0118,  0.0164,  0.0165,  ..., -0.0376, -0.0247,  0.0064]],
       device='cuda:0', dtype=torch.float64)


In [56]:
#find top-2 max values and indices
top_2_values, top_2_indices = torch.topk(attributions_sum, 2, dim=1) #top-2 values and indices for each sample in testset with class 0
top_2_indices

tensor([[ 9, 10],
        [ 7,  6],
        [ 0, 12],
        [ 6,  4],
        [ 8,  0],
        [ 7,  8],
        [10,  9],
        [ 2,  6],
        [ 5,  6],
        [ 3,  2],
        [ 2,  0],
        [ 4,  6],
        [12,  8],
        [ 3,  8],
        [ 9,  7],
        [ 4, 12],
        [ 8,  9],
        [ 1,  0],
        [ 6,  0],
        [ 6, 11],
        [ 0,  7],
        [ 3, 11],
        [ 8,  9],
        [ 7,  1],
        [ 5,  2],
        [12,  0],
        [ 8,  9],
        [ 5,  8],
        [10,  5],
        [10,  6],
        [ 9, 10],
        [ 1,  0],
        [ 7,  2],
        [ 5,  4],
        [10,  9],
        [ 8,  1],
        [ 7, 11],
        [ 3,  1],
        [10,  9],
        [10,  6],
        [ 8,  9],
        [ 8,  9],
        [ 3,  6],
        [ 5,  1],
        [ 5,  8],
        [10, 11],
        [ 9,  8],
        [11,  7],
        [ 4,  5],
        [ 6,  3],
        [ 7,  2],
        [11, 12],
        [ 6,  5],
        [ 4, 10],
        [11, 12],
        [ 

In [57]:
#final count of important positions
from collections import Counter
top_indices = [int(item) for sublist in top_2_indices for item in sublist]
count = Counter(top_indices).most_common()
count


[(6, 43),
 (7, 34),
 (8, 29),
 (5, 28),
 (9, 23),
 (4, 21),
 (1, 19),
 (10, 18),
 (2, 17),
 (3, 16),
 (11, 15),
 (0, 14),
 (12, 7)]